In [1]:
import pandas as pd
import numpy as np
import os
import gdown
import json
import re

In [2]:
# Función para extraer el dataset y guardar una copia
def extract_datsets(id_google, file_name):
    # Enlace directo al archivo en Google Drive (asegúrate de que sea el ID correcto del enlace de descarga)
    dataset_g_drive = f"https://drive.google.com/uc?id={id_google}"
    response = gdown.download(dataset_g_drive, output=f"data/original/{file_name}", quiet=True)
    df = pd.read_csv(response)
    return df

In [3]:
id_mov_google = "1Rp7SNuoRnmdoQMa5LWXuK4i7W1ILblYb"
file_name_mov = "movies_dataset.csv"

df_pre_mov = extract_datsets(id_mov_google, file_name_mov)

C:\Users\Cistelsa\AppData\Local\Temp\ipykernel_17944\2916470951.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(response)


In [4]:
id_cre_google = "1lMGJUWVVVRPO00ZWqzEJZIxFhSqtfmAB"
file_name_cre = "credits.csv"

df_pre_cre = extract_datsets(id_cre_google, file_name_cre)

In [5]:
df_pre_mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
df_pre_cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#### Función para agregar los IDs del dataframe `df_mov` y `df_cre` a una lista y guardarlos en un dataset `id_mov.csv` y `id_cre.csv` respectivamente, esto para comparar cada vez que se ejecuta si se han agregado peliculas nuevas al dataset original o no. Trabaja de forma incremental y de forma automatizada.

In [8]:
def actualizar_csv_con_nuevos_datos(df_pre, archivo_csv, columna_id):
    # Lista de ID que se quiere guardar en el archivo CSV para comparar con el dataset original
    df_id = list(df_pre[columna_id].copy())

    # Leer el archivo CSV existente (si existe) y cargar su contenido en un DataFrame de pandas
    if not os.path.exists(archivo_csv):
        lista_existente = pd.DataFrame(columns=[columna_id])
    else:
        lista_existente = pd.read_csv(archivo_csv)

    # Convertir la columna "id" del DataFrame en un conjunto para mejorar la eficiencia de búsqueda
    ids_existente_set = set(lista_existente[columna_id])

    # Obtener los nuevos datos que no están presentes en el archivo CSV existente teniendo en cuenta que es un set() y no se ingresan ids repetidos
    nuevos_datos = list(set(df_id) - ids_existente_set)

    # Crear un DataFrame con los nuevos datos
    nuevos_datos_df = pd.DataFrame(nuevos_datos, columns=[columna_id])

    # Concatenar el DataFrame existente con los nuevos datos
    lista_actualizada = pd.concat([lista_existente, nuevos_datos_df], ignore_index=True)

    # Filtrar el DataFrame df_pre para obtener solo las filas que tienen valores únicos en la columna "id"
    df_nuevos = df_pre[~df_pre[columna_id].isin(ids_existente_set)]

    # Guardar el DataFrame actualizado en el archivo CSV
    lista_actualizada.to_csv(archivo_csv, index=False)

    return df_nuevos

In [9]:
# Uso de la función actualizar_csv_con_nuevos_datos() con los parámetros apropiados
archivo_mov_csv = "data/original/id_mov.csv"
# 'columna_id' varible general usada en varias funciones
columna_id = "id"

df_mov = actualizar_csv_con_nuevos_datos(df_pre_mov, archivo_mov_csv, columna_id)

In [10]:
df_mov.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


#### Función para detectar errores los cuales no permitan realizar las tareas de ETL

In [11]:
def verificar_errores(df, archivo_csv, columna_id):
    # Copiamos la columna id para mantener el valor original
    df["original_id"] = df[columna_id]
    # Identificar las filas con tipos de dato no válidos en la columna "id"
    # Por ejemplo, si esperamos que "id" contenga solo valores numéricos (int o float):
    df[columna_id] = pd.to_numeric(df[columna_id], errors="coerce")
    invalid_rows = df[columna_id].isnull()
    df_anomalies = df[invalid_rows].copy()  # Utilizamos .copy() para asegurarnos de tener una copia independiente

    # Leer el archivo CSV existente (si existe) y cargar su contenido en un DataFrame de pandas
    if not os.path.exists(archivo_csv):
        return False, df_anomalies  # El archivo CSV no existe, no hay duplicados

    # Cargar el archivo CSV en un DataFrame
    df_existente = pd.read_csv(archivo_csv)

    # Convertir la columna "id" del DataFrame existente en un conjunto para mejorar la eficiencia de búsqueda
    ids_existente_set = set(df_existente[columna_id])

    # Reemplazar los valores NaN en la columna "id" de df_anomalies por los valores originales en formato string
    df_anomalies.loc[invalid_rows, columna_id] = df_anomalies.loc[invalid_rows, "original_id"]

    # Verificar si los registros del DataFrame actual (df_anomalies) están duplicados con los registros existentes
    duplicados = df_anomalies[columna_id].isin(ids_existente_set).any()

    # Eliminar la columna "original_id" que ya no es necesaria
    df_anomalies.drop("original_id", axis=1, inplace=True)

    return duplicados, df_anomalies

In [12]:
# Uso de la función para verificar si hay duplicados antes de guardar los datos en el archivo CSV
archivo_csv = "data/anomalies/movies_dataset_anomalies.csv"

hay_duplicados, df_anomalies = verificar_errores(df_mov, archivo_csv, columna_id)

if hay_duplicados:
    print("Hay registros duplicados en el DataFrame actual y el archivo CSV existente.")
    df_mov = df_mov.drop(df_anomalies.index)
else:
    print("No hay registros duplicados. Guardando datos anómalos en el archivo CSV...")
    df_anomalies.to_csv(archivo_csv, mode="a", index=False, header=not os.path.exists(archivo_csv))
    # Eliminar las filas identificadas con tipos de dato no válidos
    df_mov = df_mov.drop(df_anomalies.index)

Hay registros duplicados en el DataFrame actual y el archivo CSV existente.


In [13]:
df_anomalies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_mov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  float64
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

#### Se verifica que el archivo `movies_dataset_anomalies_solution.csv` tenga registros, ya que son las soluciones manuales por parte del area de ingeniería, cuando los datos vienen mal cargados del dataset original `movies_dataset.csv` pero pueden ser reutilizados.

In [15]:
def anomalias_solucionadas(solution_archivo_csv, df):
    # Cargar el archivo CSV en un DataFrame
    df_solucion_anomalias = pd.read_csv(f"data/anomalies/{solution_archivo_csv}")

    try:
        if len(df_solucion_anomalias) > 0:
            nuevo_df = pd.concat([df, df_solucion_anomalias])
            df_solucion_anomalias = df_solucion_anomalias.drop(df_solucion_anomalias.index)
            df_solucion_anomalias.to_csv(f"data/anomalies/{solution_archivo_csv}", index=False)
            return nuevo_df
        else:
            return df    
    except ValueError as e:
        print(f"Error: {e}")
        return df  # En caso de error, simplemente devolvemos el DataFrame original sin hacer la concatenación


In [16]:
solution_archivo_csv = "movies_dataset_anomalies_solution.csv"
df_mov = anomalias_solucionadas(solution_archivo_csv , df_mov)

In [17]:
df_mov.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,original_id
45464,False,NaN,0,[],NaN,227506.0,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,227506
45465,False,NaN,0,[],NaN,461257.0,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0,461257
0,NaN,NaN,NaN,NaN,NaN,5000001.0,NaN,NaN,NaN,- Written by Ørnås,...,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1.0,NaN
1,NaN,NaN,NaN,NaN,NaN,5000002.0,NaN,NaN,NaN,Rune Balot goes to a casino connected to the ...,...,0.0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12.0,NaN
2,NaN,NaN,NaN,NaN,NaN,5000003.0,NaN,NaN,NaN,Avalanche Sharks tells the story of a bikini ...,...,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22.0,NaN


#### Función para reemplazar la comilla sencilla ' por un espacio " " en los nombre y textos en inglés ya que afectan la lectura del formato Json

In [18]:
def remove_single_quotes_within_double_quotes(text):
    pattern = r'"([^"]*?\'[^"]*?)"'
    matches = re.findall(pattern, text)
    for match in matches:
        text = text.replace(match, match.replace("'", " "))
    return text

#### Solucionar errores del formato Json en la columna `genres`, `spoken_languages`, `belongs_to_collection`, `production_companies`, `production_countries` antes de proceder a la limpieza, en este caso reemplazamos la comilla sencilla que se encuentra en el lenguaje ingles por un espacio " " 

In [26]:
df_mov["genres"] = df_mov["genres"].astype(str)
df_mov["spoken_languages"] = df_mov["spoken_languages"].astype(str)
df_mov["belongs_to_collection"] = df_mov["belongs_to_collection"].astype(str)
df_mov["production_companies"] = df_mov["production_companies"].astype(str)
df_mov["production_countries"] = df_mov["production_countries"].astype(str)

In [27]:
# Aplicar la función remove_single_quotes_within_double_quotes() al DataFrame
df_mov["genres"] = df_mov["genres"].apply(remove_single_quotes_within_double_quotes)
df_mov["spoken_languages"] = df_mov["spoken_languages"].apply(remove_single_quotes_within_double_quotes)
df_mov["belongs_to_collection"] = df_mov["belongs_to_collection"].apply(remove_single_quotes_within_double_quotes)
df_mov["production_companies"] = df_mov["production_companies"].apply(remove_single_quotes_within_double_quotes)
df_mov["production_countries"] = df_mov["production_countries"].apply(remove_single_quotes_within_double_quotes)

#### Función de limpieza según el requerimiento del Area

In [29]:
def clean_data_mov(df_mov):
    # Convertir la columna "id" en tipo entero
    df_mov = df_mov.astype({"id": "int64"})
    # Eliminar peliculas repetidas desde la columna "id" ids repetidos
    df_mov = df_mov.drop_duplicates(subset=["id"])
    # Eliminar filas con registros vacíos en la columna "release_date"
    df_mov = df_mov.dropna(subset=["release_date"])
    # Los campos vacíos se reemplazan por "0" en las columnas "budget", "revenue", "runtime"
    df_mov = df_mov.fillna({"budget": "0", "revenue": "0", "runtime": "0"})
    # Cambiar a tipo float las columnas "budget", "revenue", "runtime", "popularity", "vote_count"
    df_mov = df_mov.astype({"budget": "float64", "revenue": "float64", "runtime": "int64", "popularity": "float64", "vote_count": "int64"})
    # Convertir la columna "release_date" a tipo datetime
    df_mov["release_date"] = pd.to_datetime(df_mov["release_date"])
    # Crear una columna "release_year" con el año unicamente
    df_mov["release_year"] = df_mov["release_date"].dt.year
    # Crear una columna "return" para saber el retorno dividiendo "revenue / budget" se modifica el resultado inf a 0
    df_mov["return"] = np.divide(df_mov["revenue"], df_mov["budget"], out=np.zeros_like(df_mov["revenue"]), where=(df_mov["budget"]!=0))
    # Se redondea la columna "return" a 2 decimales
    df_mov["return"] = round(df_mov["return"], 2)
    # Los campos vacíos se reemplazan por 0 en la columna "return"
    df_mov = df_mov.fillna({"return": 0})
    # Se redondea la columna "popularity" a 2 decimales
    df_mov["popularity"] = round(df_mov["popularity"], 2)
    # Reemplazar caracteres ' por " en la columna: "genres"
    df_mov["genres"] = df_mov["genres"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "genres" para solucionar errores del formato Json
    df_mov["genres"] = df_mov["genres"].str.replace("None", "\"\"", case=False, regex=False)
    # Reemplazar caracteres ' por " en la columna: "spoken_languages"
    df_mov["spoken_languages"] = df_mov["spoken_languages"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "spoken_languages" para solucionar errores del formato Json
    df_mov["spoken_languages"] = df_mov["spoken_languages"].str.replace("None", "\"\"", case=False, regex=False)
    # Reemplazar caracteres ' por " en la columna: "belongs_to_collection"
    df_mov["belongs_to_collection"] = df_mov["belongs_to_collection"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "belongs_to_collection" para solucionar errores del formato Json
    df_mov["belongs_to_collection"] = df_mov["belongs_to_collection"].str.replace("None", "\"\"", case=False, regex=False)
    # Agregar los corchetes al inicio y al final de los valores en la columna "belongs_to_collection" para que quede en formato Json
    df_mov["belongs_to_collection"] = "[" + df_mov["belongs_to_collection"] + "]"
    # Reemplazar caracteres ' por " en la columna: "production_companies"
    df_mov["production_companies"] = df_mov["production_companies"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "production_companies" para solucionar errores del formato Json
    df_mov["production_companies"] = df_mov["production_companies"].str.replace("None", "\"\"", case=False, regex=False)
    # Reemplazar caracteres ' por " en la columna: "production_countries"
    df_mov["production_countries"] = df_mov["production_countries"].str.replace("'", "\"", case=False, regex=False)
     # Reemplazar texto "None" por "" (espacios vacíos) en la columna "production_countries" para solucionar errores del formato Json
    df_mov["production_countries"] = df_mov["production_countries"].str.replace("None", "\"\"", case=False, regex=False)
    # Eliminar columnas: "video", "imdb_id", "adult", "original_title", "poster_path", "homepage"
    df_mov = df_mov.drop(columns=["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])
    # Organizar por columna "id" (ascendente)
    df_mov = df_mov.sort_values(["id"], ignore_index=True)
    # Reorganizar las columnas del dataframe
    df_mov = df_mov[["id", "title", "original_language", "overview", "tagline", "status", "runtime", "release_date", "release_year", "genres", "spoken_languages", "budget", "revenue", "return", "popularity", "vote_average", "vote_count", "belongs_to_collection", "production_companies", "production_countries"]]
    return df_mov

df_mov_clean = clean_data_mov(df_mov.copy())
df_mov_clean.head(15)

,id,title,original_language,overview,tagline,status,runtime,release_date,release_year,genres,spoken_languages,budget,revenue,return,popularity,vote_average,vote_count,belongs_to_collection,production_companies,production_countries
0,2,Ariel,fi,Taisto Kasurinen is a Finnish coal miner whose...,NaN,Released,69,1988-10-21,1988,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","[{""iso_639_1"": ""fi"", ""name"": ""suomi""}, {""iso_6...",0.0,0.0,0.00,3.86,7.1,44,[nan],"[{""name"": ""Villealfa Filmproduction Oy"", ""id"":...","[{""iso_3166_1"": ""FI"", ""name"": ""Finland""}]"
1,3,Shadows in Paradise,fi,"An episode in the life of Nikander, a garbage ...",NaN,Released,76,1986-10-16,1986,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...","[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",0.0,0.0,0.00,2.29,7.1,35,[nan],"[{""name"": ""Villealfa Filmproduction Oy"", ""id"":...","[{""iso_3166_1"": ""FI"", ""name"": ""Finland""}]"
2,5,Four Rooms,en,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,Released,98,1995-12-09,1995,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",4000000.0,4300000.0,1.08,9.03,6.5,539,[nan],"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
3,6,Judgment Night,en,"While racing to a boxing match, Frank, Mike, J...",Don't move. Don't whisper. Don't even breathe.,Released,110,1993-10-15,1993,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",0.0,12136938.0,0.00,5.54,6.4,79,[nan],"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}, {""iso_..."
4,11,Star Wars,en,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",Released,121,1977-05-25,1977,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",11000000.0,775398007.0,70.49,42.15,8.1,6778,"[{""id"": 10, ""name"": ""Star Wars Collection"", ""p...","[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
5,12,Finding Nemo,en,"Nemo, an adventurous young clownfish, is unexp...",There are 3.7 trillion fish in the ocean. They...,Released,100,2003-05-30,2003,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",94000000.0,940335536.0,10.00,25.50,7.6,6292,"[{""id"": 137697, ""name"": ""Finding Nemo Collecti...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
6,13,Forrest Gump,en,A man with a low IQ has accomplished great thi...,"The world will never be the same, once you've ...",Released,142,1994-07-06,1994,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",55000000.0,677945399.0,12.33,48.31,8.2,8147,[nan],"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
7,14,American Beauty,en,"Lester Burnham, a depressed suburban father in...",Look closer.,Released,122,1999-09-15,1999,"[{""id"": 18, ""name"": ""Drama""}]","[{""iso_639_1"": ""en"", ""name"": ""English""}]",15000000.0,356296601.0,23.75,20.73,7.9,3438,[nan],"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
8,15,Citizen Kane,en,"Newspaper magnate, Charles Foster Kane is take...",It's Terrific!,Released,119,1941-04-30,1941,"[{""id"": 9648, ""name"": ""Mystery""}, {""id"": 18, ""...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",839727.0,23217674.0,27.65,15.81,8.0,1244,[nan],"[{""name"": ""RKO Radio Pictures"", ""id"": 6}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o..."
9,16,Dancer in the Dark,en,"Selma, a Czech imm

In [30]:
df_mov_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45349 entries, 0 to 45348
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45349 non-null  int64         
 1   title                  45349 non-null  object        
 2   original_language      45335 non-null  object        
 3   overview               44408 non-null  object        
 4   tagline                20388 non-null  object        
 5   status                 45269 non-null  object        
 6   runtime                45349 non-null  int64         
 7   release_date           45349 non-null  datetime64[ns]
 8   release_year           45349 non-null  int64         
 9   genres                 45349 non-null  object        
 10  spoken_languages       45349 non-null  object        
 11  budget                 45349 non-null  float64       
 12  revenue                45349 non-null  float64       
 13  r

#### Fin del ETL para el datset `Movies_dataset.csv`

In [31]:
df_mov_clean.to_csv("data/beta/movies_dataset_beta.csv", index=False)

#### Inicio ETL dataset `credits.csv`

In [32]:
df_pre_cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [33]:
df_pre_cre.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [34]:
# Uso de la función actualizar_csv_con_nuevos_datos() con los parámetros apropiados
archivo_cre_csv = "data/original/id_cre.csv"

df_cre = actualizar_csv_con_nuevos_datos(df_pre_cre, archivo_cre_csv, columna_id)

In [35]:
# Llamamos la función anomalias_solucionadas() para cargarlas nuevamente si existen
solution_cre_archivo_csv = "credits_dataset_anomalies_solution.csv"
df_cre = anomalias_solucionadas(solution_cre_archivo_csv , df_cre)

In [36]:
# Uso de la función para verificar si hay duplicados antes de guardar los datos en el archivo CSV
archivo_cre_csv = "data/anomalies/credits_dataset_anomalies.csv"

hay_duplicados_cre, df_anomalies_cre = verificar_errores(df_cre, archivo_cre_csv, columna_id)

if hay_duplicados_cre:
    print("Hay registros duplicados en el DataFrame actual y el archivo CSV existente.")
else:
    print("No hay registros duplicados. Guardando datos anómalos en el archivo CSV...")
    df_anomalies_cre.to_csv(archivo_csv, mode='a', index=False, header=not os.path.exists(archivo_csv))
    # Eliminar las filas identificadas con tipos de dato no válidos
    df_cre = df_cre.drop(df_anomalies_cre.index)

No hay registros duplicados. Guardando datos anómalos en el archivo CSV...


#### Solucionar errores del formato Json en la columna `crew` antes de proceder a la limpieza, en este caso reemplazamos la comilla sencilla que se encuentra en el lenguaje ingles por un espacio " " 

In [37]:
# Aplicar la función remove_single_quotes_within_double_quotes() al DataFrame
df_cre["crew"] = df_cre["crew"].apply(remove_single_quotes_within_double_quotes)
df_cre["cast"] = df_cre["cast"].apply(remove_single_quotes_within_double_quotes)

#### Función para ETL dataframe `df_cre`

In [38]:
def clean_data_cre(df_cre):
    # Convertimos la columna "id" a entero
    df_cre = df_cre.astype({"id": "int64"})
    # Eliminar duplicados en la columna "id"
    df_cre = df_cre.drop_duplicates(subset=["id"])
    # Reorganizar las columnas del dataframe
    df_cre = df_cre[["id", "cast", "crew"]]
    # Reemplazar caracteres ' por " en la columna: "crew"
    df_cre["cast"] = df_cre["cast"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "crew" para solucionar errores del formato Json
    df_cre["cast"] = df_cre["cast"].str.replace("None", "\"\"", case=False, regex=False)
    # Reemplazar caracteres ' por " en la columna: "crew"
    df_cre["crew"] = df_cre["crew"].str.replace("'", "\"", case=False, regex=False)
    # Reemplazar texto "None" por "" (espacios vacíos) en la columna "crew" para solucionar errores del formato Json
    df_cre["crew"] = df_cre["crew"].str.replace("None", "\"\"", case=False, regex=False)
    # Organizar columna por "id" (ascendente)
    df_cre = df_cre.sort_values(["id"], ignore_index=True)
    return df_cre

df_cre_clean = clean_data_cre(df_cre.copy())
df_cre_clean.head(15)

,id,cast,crew
0,2,"[{""cast_id"": 3, ""character"": ""Taisto Olavi Kas...","[{""credit_id"": ""52fe420dc3a36847f800001f"", ""de..."
1,3,"[{""cast_id"": 5, ""character"": ""Nikander"", ""cred...","[{""credit_id"": ""52fe420dc3a36847f8000077"", ""de..."
2,5,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800011b"", ""de..."
3,6,"[{""cast_id"": 7, ""character"": ""Frank Wyatt"", ""c...","[{""credit_id"": ""52fe420dc3a36847f800023d"", ""de..."
4,11,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
5,12,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
6,13,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800072d"", ""de..."
7,14,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f80007c5"", ""de..."
8,15,"[{""cast_id"": 5, ""character"": ""Charles Foster K...","[{""credit_id"": ""52fe420ec3a36847f8000885"", ""de..."
9,16,"[{""cast_id"": 33, ""character"": ""Selma Jezkova"",...","[{""credit_id"": ""52fe420ec3a36847f8000981"", ""de..."


#### Fin ETL dataset `credits.csv`

In [39]:
df_cre_clean.to_csv("data/beta/credits_beta.csv", index=False)